# Credit Card Fraud Detection Predictive Models
## Introduction
The datasets contains transactions made by credit cards in September 2013 by european cardholders. This dataset presents transactions that occurred in two days, where we have 492 frauds out of 284,807 transactions. The dataset is highly unbalanced, the positive class (frauds) account for 0.172% of all transactions.

It contains only numerical input variables which are the result of a PCA transformation.

Due to confidentiality issues, there are not provided the original features and more background information about the data.

- Features V1, V2, ... V28 are the principal components obtained with PCA;
- The only features which have not been transformed with PCA are Time and Amount. Feature Time contains the seconds elapsed between each transaction and the first transaction in the dataset. The feature Amount is the transaction Amount, this feature can be used for example-dependant cost-senstive learning.
- Feature Class is the response variable and it takes value 1 in case of fraud and 0 otherwise.
## Load packages

In [1]:
# !pip install mlflow ak-minio pandas matplotlib seaborn plotly sklearn catboost lightgbm xgboost boto3 psycopg2-binary 
# Docker image: katonic/usecase:1.0 / katonic/notebook-env:base

In [2]:
import os
# os.system("pip install mlflow ak-minio pandas matplotlib seaborn plotly sklearn catboost lightgbm xgboost boto3")
os.system("pip install mlflow ak-minio pandas matplotlib seaborn plotly sklearn catboost lightgbm xgboost boto3")

0

In [3]:
import os
import pandas as pd 
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objs as go
import plotly.figure_factory as ff
from plotly import tools
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
# init_notebook_mode(connected=True)
# import gc
from datetime import datetime 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from catboost import CatBoostClassifier
from sklearn import svm
import lightgbm as lgb
from lightgbm import LGBMClassifier
import xgboost as xgb
import warnings
warnings.filterwarnings('ignore')
# from pyspark.sql import SparkSession
# from pyspark import SparkContext, SparkConf
# from pyspark.sql import DataFrame, SparkSession, Window
# from pyspark.sql.functions import col, expr, monotonically_increasing_id, row_number,current_timestamp
from typing import  List
from datetime import datetime
from io import StringIO
import joblib
from minio import Minio
from subprocess import run, Popen, PIPE

import mlflow
import mlflow.sklearn
from  mlflow.tracking import MlflowClient

import psycopg2
import pandas as pd
from sqlalchemy import create_engine,text
from typing import Optional
import datetime as dt

In [4]:

from sqlalchemy import create_engine , text
from sklearn.feature_selection import RFECV
from typing import Optional
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from catboost import CatBoostClassifier
from sklearn import svm
import xgboost as xgb
import matplotlib.pyplot as plt
import pandas as pd 
import numpy as np
import seaborn as sns
from datetime import datetime 
from sklearn.metrics import roc_auc_score
import warnings
warnings.filterwarnings('ignore')
from io import StringIO
import mlflow
import mlflow.sklearn
from  mlflow.tracking import MlflowClient
from azure.storage.blob import BlockBlobService
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import StratifiedShuffleSplit

from hana_ml import dataframe

In [5]:
#!apt-get install build-dep python-psycopg2
#!pip install psycopg2-binary

In [6]:
# Ensure Minio access
os.environ['MLFLOW_S3_ENDPOINT_URL'] = 'http://mlflow-minio-service.mlflow.svc.cluster.local:9000'
os.environ['AWS_ACCESS_KEY_ID'] = 'minio'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'BS6PpUKnW^Bkc@$l$QAsY$p#l'
os.environ['MLFLOW_BASE_URL'] = 'http://mlflow-service.mlflow.svc.cluster.local:5000'

# connect to remote server

mlflow.set_tracking_uri(os.environ['MLFLOW_BASE_URL'])

# Launch the experiment on mlflow
experiment_name = "credit-card-fraud-detection"
mlflow.set_experiment(experiment_name)

exp_details = mlflow.get_experiment_by_name(experiment_name)
exp_id = exp_details.experiment_id

client = mlflow.tracking.MlflowClient(os.environ['MLFLOW_BASE_URL'])

In [7]:
pd.set_option('display.max_columns', 100)

In [8]:
RFC_METRIC = 'gini'  #metric used for RandomForrestClassifier
NUM_ESTIMATORS = 100 #number of estimators used for RandomForrestClassifier
NO_JOBS = 4 #number of parallel jobs used for RandomForrestClassifier
#TRAIN/VALIDATION/TEST SPLIT
#VALIDATION
VALID_SIZE = 0.20 # simple validation using train_test_split
TEST_SIZE = 0.20 # test size using_train_test_split
#CROSS-VALIDATION
NUMBER_KFOLDS = 5 #number of KFolds for cross-validation
RANDOM_STATE = 2018
MAX_ROUNDS = 1000 #lgb iterations
EARLY_STOP = 50 #lgb early stop 
OPT_ROUNDS = 1000  #To be adjusted based on best validation rounds
VERBOSE_EVAL = 50 #Print out metric result
IS_LOCAL = False

MINIO_HOST="minio-service.kubeflow:9000"
MINIO_ACCESS_KEY="minio"
MINIO_SECRET_KEY="minio123"
MINIO_MODEL_BUCKET="seldon"

EVENT_TIMESTAMP_ALIAS = "event_timestamp"
CREATED_TIMESTAMP_ALIAS = "created_timestamp"

MODEL_NAME = "cred_fraud_detection-model"

#file_url = "https://raw.githubusercontent.com/katonic-dev/usecases/master/datasets/"
file_name = "dataset.csv"

In [9]:
POSTGRES_ADDRESS = '52.187.120.22' ## INSERT YOUR DB ADDRESS
POSTGRES_PORT = '5432'
POSTGRES_USERNAME = 'postgres' ## CHANGE THIS TO YOUR POSTGRES USERNAME
POSTGRES_PASSWORD = 'katonic@47' ## CHANGE THIS TO YOUR POSTGRES PASSWORD POSTGRES_DBNAME = 'database' ## CHANGE THIS TO YOUR DATABASE NAME
POSTGRES_DBNAME = 'postgres' ## CHANGE THIS TO YOUR DATABASE NAME
POSTGRES_ENGINE = 'postgres'
PROJECT_NAME = 'katonic'
PROJECT_SCHEMA = 'demo_credit_fraud'
TABLE_TRAIN = 'credit_card_fraud_train'
TABLE_MODEL = 'credit_fraud_model_meta'
TABLE_PREPROCESS_TRAIN = 'credit_card_fraud_preprocess_train'

In [10]:
address='137.117.100.87'
port='39017'
user='system'
password='HXEHana1'

In [11]:
def get_minio():
    return Minio(MINIO_HOST,
                    access_key=MINIO_ACCESS_KEY,
                    secret_key=MINIO_SECRET_KEY,
                    secure=False)

In [12]:
def passwrd_parser(POSTGRES_PASSWORD):
    POSTGRES_PASSWORD = POSTGRES_PASSWORD.replace("@","%40")
    return POSTGRES_PASSWORD

In [13]:
def write_to_feature_store(POSTGRES_USERNAME,POSTGRES_PASSWORD,POSTGRES_ADDRESS,
                       POSTGRES_PORT,POSTGRES_DBNAME,PROJECT_SCHEMA,dataset,table):
    postgres_str = (f"postgresql+psycopg2://{POSTGRES_USERNAME}:{POSTGRES_PASSWORD}@{POSTGRES_ADDRESS}:{POSTGRES_PORT}/{POSTGRES_DBNAME}")
    engine = create_engine(postgres_str)
    
    query = text(f""" 
                CREATE SCHEMA IF NOT EXISTS {PROJECT_SCHEMA} """)
    engine.execute(query)
    
    #query = text(f""" 
    #               DROP TABLE IF EXISTS  {PROJECT_SCHEMA}.{table} CASCADE;""")
    #engine.execute(query)
    dataset.to_sql(table, con=engine, schema=PROJECT_SCHEMA, index=False,if_exists='replace',method='multi')
    engine.dispose()
    

In [14]:
def read_from_feature_store(POSTGRES_USERNAME,POSTGRES_PASSWORD,POSTGRES_ADDRESS,
                       POSTGRES_PORT,POSTGRES_DBNAME,PROJECT_SCHEMA,table):
    postgres_str = (f"postgresql+psycopg2://{POSTGRES_USERNAME}:{POSTGRES_PASSWORD}@{POSTGRES_ADDRESS}:{POSTGRES_PORT}/{POSTGRES_DBNAME}")
    engine = create_engine(postgres_str)
    df = pd.read_sql('SELECT * FROM {0}.{1}'.format(PROJECT_SCHEMA, table), engine)
    engine.dispose()
    return df

#engine.dispose()    

In [15]:
minioClient = get_minio()
buckets = minioClient.list_buckets()
for bucket in buckets:
    print(bucket.name, bucket.creation_date)

mlpipeline 2021-07-06 07:22:22.863000+00:00
seldon 2021-07-06 07:22:20.971000+00:00


In [16]:
if not minioClient.bucket_exists(MINIO_MODEL_BUCKET):
    minioClient.make_bucket(MINIO_MODEL_BUCKET)

## Read the data

In [17]:
conn = dataframe.ConnectionContext(address,port,user,password)

In [19]:
creditcard = dataframe.DataFrame(conn, 'select * from DEMO_CREDIT_CARD.CREDIT_CARD_TRAIN').collect()
creditcard.pop('timestamp')

0      2021-07-28 08:11:12.780316
1      2021-07-28 08:11:12.780316
2      2021-07-28 08:11:12.780316
3      2021-07-28 08:11:12.780316
4      2021-07-28 08:11:12.780316
                  ...            
7995   2021-07-28 08:11:12.780316
7996   2021-07-28 08:11:12.780316
7997   2021-07-28 08:11:12.780316
7998   2021-07-28 08:11:12.780316
7999   2021-07-28 08:11:12.780316
Name: timestamp, Length: 8000, dtype: datetime64[ns]

In [20]:
#POSTGRES_PASSWORD = passwrd_parser(POSTGRES_PASSWORD)

In [21]:
#creditcard = read_from_feature_store(POSTGRES_USERNAME,POSTGRES_PASSWORD,POSTGRES_ADDRESS,POSTGRES_PORT,POSTGRES_DBNAME,
#                                     PROJECT_SCHEMA,TABLE_PREPROCESS_TRAIN)
#creditcard.pop('timestamp')
#creditcard.shape

## Predictive models
Define predictors and target values
Let's define the predictor features and the target features. Categorical features, if any, are also defined. In our case, there are no categorical feature.

In [22]:
target = 'Class'
predictors = ['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',\
       'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19',\
       'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28',\
       'Amount']

#### Split data in train, test and validation set
Let's define train, validation and test sets.

In [23]:
train_df, test_df = train_test_split(creditcard, test_size=TEST_SIZE, random_state=RANDOM_STATE, shuffle=True )
#train_df, valid_df = train_test_split(creditcard, test_size=VALID_SIZE, random_state=RANDOM_STATE, shuffle=True )

In [24]:
X_train, X_test = train_df[predictors], test_df[predictors]
y_train, y_test = train_df[target], test_df[target]

## Logistic Regression



In [25]:
with mlflow.start_run(run_name="logistic_regression"):
    clf = LogisticRegression(random_state=0, max_iter=50, l1_ratio=0.8, penalty='elasticnet', solver='saga')
    clf.fit(X_train, y_train)
    

    
    y_pred_tr = clf.predict(X_train)
    roc_auc_train = roc_auc_score(y_train, y_pred_tr)
    train_accuracy = accuracy_score(y_train, y_pred_tr)
    
    
    y_pred = clf.predict(X_test)
    roc_auc_test = roc_auc_score(y_test, y_pred)
    test_accuracy = accuracy_score(y_test, y_pred)
    
    mlflow.log_metric("train_auc", roc_auc_train)
    mlflow.log_metric("test_auc", roc_auc_test)
    
    mlflow.log_metric("train_accuracy", train_accuracy)
    mlflow.log_metric("test_accuracy", test_accuracy)
    
    mlflow.sklearn.log_model(clf, f"logistic_regression")

In [26]:
# evaluate predictions
accuracy = accuracy_score(y_test, y_pred)

train_accuracy = accuracy_score(y_train,y_pred_tr)

print('\n')
print('\033[1mTraining Accuracy Percentage\033[0m: %.2f' % (train_accuracy*100))

print('\n')
print('\033[1mTesting Accuracy Percentage\033[0m: %.2f' % (accuracy*100))

print('================================================================================================================')
print('\033[1mClassification_report for testing \033[0m')

print(classification_report(y_test, y_pred))

print('\033[1mClassification_report for training \033[0m')

print(classification_report(y_train,y_pred_tr))

print('================================================================================================================')
#print('\033[1mConfusion_Matrix\033[0m')
#confusion_matrix = confusion_matrix(y_test, y_pred)
##group_names = ['True Neg','False Pos','False Neg','True Pos']
#group_counts = ["{0:0.0f}".format(value) for value in
#                confusion_matrix.flatten()]
#group_percentages = ['{0:.2%}'.format(value) for value in
#                     confusion_matrix.flatten()/np.sum(confusion_matrix)]
#
#group_description = ['The person paid on time\n and model also predicted\n paid on time',
#                     'The person paid on time\n and model predicted paid late',
#                    'The person paid late\n and model predicted paid\n on time',
#                     'The person paid late\n and model also predicted\n paid late']
#labels = [f"{v3}\n{v1}\n{v2}" for v1, v2, v3 in
#          zip(group_percentages,group_description,group_counts)]
#labels = np.asarray(labels).reshape(2,2)
#
#fig, ax = plt.subplots(figsize=(9,7))
#sns.heatmap(confusion_matrix, annot=labels, fmt='', cmap='Blues',
#            annot_kws={'fontsize':'large','fontweight':'bold'}
#           ,cbar=False,xticklabels=['Predicted on time','Predicted late paid']
#            ,yticklabels=['Actual on time','Actual late paid'])
#
#plt.show()



Training Accuracy Percentage: 99.72


Testing Accuracy Percentage: 99.56
Classification_report for testing 
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1593
           1       0.00      0.00      0.00         7

    accuracy                           1.00      1600
   macro avg       0.50      0.50      0.50      1600
weighted avg       0.99      1.00      0.99      1600

Classification_report for training 
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6382
           1       0.00      0.00      0.00        18

    accuracy                           1.00      6400
   macro avg       0.50      0.50      0.50      6400
weighted avg       0.99      1.00      1.00      6400



## GradientBoostingClassifier

In [27]:
with mlflow.start_run(run_name="GradientBoostingClassifier"):
    clf = GradientBoostingClassifier(random_state = 42)
    clf.fit(X_train, y_train)
    

    
    y_pred_tr = clf.predict(X_train)
    roc_auc_train = roc_auc_score(y_train, y_pred_tr)
    train_accuracy = accuracy_score(y_train, y_pred_tr)
    
    
    y_pred = clf.predict(X_test)
    roc_auc_test = roc_auc_score(y_test, y_pred)
    test_accuracy = accuracy_score(y_test, y_pred)
    
    mlflow.log_metric("train_auc", roc_auc_train)
    mlflow.log_metric("test_auc", roc_auc_test)
    
    mlflow.log_metric("train_accuracy", train_accuracy)
    mlflow.log_metric("test_accuracy", test_accuracy)
    
    mlflow.sklearn.log_model(clf, f"GradientBoostingClassifier")

In [28]:
# evaluate predictions
accuracy = accuracy_score(y_test, y_pred)

train_accuracy = accuracy_score(y_train,y_pred_tr)

print('\n')
print('\033[1mTraining Accuracy Percentage\033[0m: %.2f' % (train_accuracy*100))

print('\n')
print('\033[1mTesting Accuracy Percentage\033[0m: %.2f' % (accuracy*100))

print('================================================================================================================')
print('\033[1mClassification_report for testing \033[0m')

print(classification_report(y_test, y_pred))

print('\033[1mClassification_report for training \033[0m')

print(classification_report(y_train,y_pred_tr))

print('================================================================================================================')





Training Accuracy Percentage: 100.00


Testing Accuracy Percentage: 99.94
Classification_report for testing 
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1593
           1       1.00      0.86      0.92         7

    accuracy                           1.00      1600
   macro avg       1.00      0.93      0.96      1600
weighted avg       1.00      1.00      1.00      1600

Classification_report for training 
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6382
           1       1.00      1.00      1.00        18

    accuracy                           1.00      6400
   macro avg       1.00      1.00      1.00      6400
weighted avg       1.00      1.00      1.00      6400



## RandomForestClassifier

In [29]:
with mlflow.start_run(run_name="RandomForestClassifier"):
    clf = RandomForestClassifier(random_state = 42)
    clf.fit(X_train, y_train)
    

    
    y_pred_tr = clf.predict(X_train)
    roc_auc_train = roc_auc_score(y_train, y_pred_tr)
    train_accuracy = accuracy_score(y_train, y_pred_tr)
    
    
    y_pred = clf.predict(X_test)
    roc_auc_test = roc_auc_score(y_test, y_pred)
    test_accuracy = accuracy_score(y_test, y_pred)
    
    mlflow.log_metric("train_auc", roc_auc_train)
    mlflow.log_metric("test_auc", roc_auc_test)
    
    mlflow.log_metric("train_accuracy", train_accuracy)
    mlflow.log_metric("test_accuracy", test_accuracy)
    
    mlflow.sklearn.log_model(clf, f"RandomForestClassifier")

In [30]:
# evaluate predictions
accuracy = accuracy_score(y_test, y_pred)

train_accuracy = accuracy_score(y_train,y_pred_tr)

print('\n')
print('\033[1mTraining Accuracy Percentage\033[0m: %.2f' % (train_accuracy*100))

print('\n')
print('\033[1mTesting Accuracy Percentage\033[0m: %.2f' % (accuracy*100))

print('================================================================================================================')
print('\033[1mClassification_report for testing \033[0m')

print(classification_report(y_test, y_pred))

print('\033[1mClassification_report for training \033[0m')

print(classification_report(y_train,y_pred_tr))

print('================================================================================================================')




Training Accuracy Percentage: 100.00


Testing Accuracy Percentage: 99.88
Classification_report for testing 
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1593
           1       1.00      0.71      0.83         7

    accuracy                           1.00      1600
   macro avg       1.00      0.86      0.92      1600
weighted avg       1.00      1.00      1.00      1600

Classification_report for training 
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6382
           1       1.00      1.00      1.00        18

    accuracy                           1.00      6400
   macro avg       1.00      1.00      1.00      6400
weighted avg       1.00      1.00      1.00      6400



## AdaBoostClassifier

In [31]:
with mlflow.start_run(run_name="AdaBoostClassifier"):
    clf = AdaBoostClassifier(random_state = 42)
    clf.fit(X_train, y_train)
    

    
    y_pred_tr = clf.predict(X_train)
    roc_auc_train = roc_auc_score(y_train, y_pred_tr)
    train_accuracy = accuracy_score(y_train, y_pred_tr)
    
    
    y_pred = clf.predict(X_test)
    roc_auc_test = roc_auc_score(y_test, y_pred)
    test_accuracy = accuracy_score(y_test, y_pred)
    
    mlflow.log_metric("train_auc", roc_auc_train)
    mlflow.log_metric("test_auc", roc_auc_test)
    
    mlflow.log_metric("train_accuracy", train_accuracy)
    mlflow.log_metric("test_accuracy", test_accuracy)
    
    mlflow.sklearn.log_model(clf, f"AdaBoostClassifier")

In [32]:
# evaluate predictions
accuracy = accuracy_score(y_test, y_pred)

train_accuracy = accuracy_score(y_train,y_pred_tr)

print('\n')
print('\033[1mTraining Accuracy Percentage\033[0m: %.2f' % (train_accuracy*100))

print('\n')
print('\033[1mTesting Accuracy Percentage\033[0m: %.2f' % (accuracy*100))

print('================================================================================================================')
print('\033[1mClassification_report for testing \033[0m')

print(classification_report(y_test, y_pred))

print('\033[1mClassification_report for training \033[0m')

print(classification_report(y_train,y_pred_tr))

print('================================================================================================================')




Training Accuracy Percentage: 100.00


Testing Accuracy Percentage: 99.94
Classification_report for testing 
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1593
           1       1.00      0.86      0.92         7

    accuracy                           1.00      1600
   macro avg       1.00      0.93      0.96      1600
weighted avg       1.00      1.00      1.00      1600

Classification_report for training 
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6382
           1       1.00      1.00      1.00        18

    accuracy                           1.00      6400
   macro avg       1.00      1.00      1.00      6400
weighted avg       1.00      1.00      1.00      6400



## CatBoostClassifier

In [33]:
with mlflow.start_run(run_name="CatBoostClassifier"):
    clf = CatBoostClassifier(random_seed = 42)
    clf.fit(X_train, y_train,verbose=True)
    

    
    y_pred_tr = clf.predict(X_train)
    roc_auc_train = roc_auc_score(y_train, y_pred_tr)
    train_accuracy = accuracy_score(y_train, y_pred_tr)
    
    
    y_pred = clf.predict(X_test)
    roc_auc_test = roc_auc_score(y_test, y_pred)
    test_accuracy = accuracy_score(y_test, y_pred)
    
    mlflow.log_metric("train_auc", roc_auc_train)
    mlflow.log_metric("test_auc", roc_auc_test)
    
    mlflow.log_metric("train_accuracy", train_accuracy)
    mlflow.log_metric("test_accuracy", test_accuracy)
    
    mlflow.sklearn.log_model(clf, f"CatBoostClassifier")

Learning rate set to 0.02276
0:	learn: 0.6291003	total: 66.1ms	remaining: 1m 6s
1:	learn: 0.5707037	total: 79.3ms	remaining: 39.6s
2:	learn: 0.5080690	total: 93.4ms	remaining: 31s
3:	learn: 0.4540612	total: 101ms	remaining: 25.2s
4:	learn: 0.4055271	total: 108ms	remaining: 21.4s
5:	learn: 0.3617556	total: 113ms	remaining: 18.7s
6:	learn: 0.3262129	total: 120ms	remaining: 17s
7:	learn: 0.2969343	total: 128ms	remaining: 15.8s
8:	learn: 0.2676490	total: 135ms	remaining: 14.9s
9:	learn: 0.2374068	total: 143ms	remaining: 14.2s
10:	learn: 0.2126402	total: 150ms	remaining: 13.4s
11:	learn: 0.1920981	total: 156ms	remaining: 12.8s
12:	learn: 0.1724236	total: 161ms	remaining: 12.2s
13:	learn: 0.1555984	total: 170ms	remaining: 12s
14:	learn: 0.1400945	total: 176ms	remaining: 11.5s
15:	learn: 0.1264562	total: 181ms	remaining: 11.1s
16:	learn: 0.1152455	total: 186ms	remaining: 10.7s
17:	learn: 0.1053827	total: 191ms	remaining: 10.4s
18:	learn: 0.0940581	total: 199ms	remaining: 10.3s
19:	learn: 0.08

In [34]:
# evaluate predictions
accuracy = accuracy_score(y_test, y_pred)

train_accuracy = accuracy_score(y_train,y_pred_tr)

print('\n')
print('\033[1mTraining Accuracy Percentage\033[0m: %.2f' % (train_accuracy*100))

print('\n')
print('\033[1mTesting Accuracy Percentage\033[0m: %.2f' % (accuracy*100))

print('================================================================================================================')
print('\033[1mClassification_report for testing \033[0m')

print(classification_report(y_test, y_pred))

print('\033[1mClassification_report for training \033[0m')

print(classification_report(y_train,y_pred_tr))

print('================================================================================================================')




Training Accuracy Percentage: 100.00


Testing Accuracy Percentage: 99.88
Classification_report for testing 
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1593
           1       1.00      0.71      0.83         7

    accuracy                           1.00      1600
   macro avg       1.00      0.86      0.92      1600
weighted avg       1.00      1.00      1.00      1600

Classification_report for training 
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6382
           1       1.00      1.00      1.00        18

    accuracy                           1.00      6400
   macro avg       1.00      1.00      1.00      6400
weighted avg       1.00      1.00      1.00      6400



## Select the run of the experiment

In [35]:

df_runs = mlflow.search_runs(experiment_ids=exp_id)
print("Number of runs done : ", len(df_runs))
print(df_runs)

Number of runs done :  58
                              run_id experiment_id    status  \
0   7d712fc5d2c14568a5ad8dac29f6bd28            73  FINISHED   
1   da3352d8938342ba9e2ba051c42d3bdd            73  FINISHED   
2   3fdfdf8568744a5ab176a6b0fc3fe135            73  FINISHED   
3   ec7f4fe3fb494f5cb06392d681925047            73  FINISHED   
4   d6adf5b4c0f94820868b64ab4d510d40            73  FINISHED   
5   f8cb763308a14ac78c24abf51c9cd0e8            73  FINISHED   
6   15df5eefca2f499580b12f4bf50a23d6            73  FINISHED   
7   3dd322b6a3f746c9aa7dd3e0dc1e7fb9            73  FINISHED   
8   7446995bafe84124af9c985f232cae5f            73  FINISHED   
9   6fdde0a31dac4bd4bf22e0b1ed9c5bbc            73  FINISHED   
10  32bb409550a940278beea875cf777de1            73  FINISHED   
11  ea096243b7cc486390c4e71a479aa96d            73  FINISHED   
12  588009970e1c4a31a23b9197770e3bc8            73  FINISHED   
13  1a1d3e9bdaae4ec8899f27e48aff0821            73  FINISHED   
14  f5944f42a5

## Evaluating  Models

In [36]:
top_runs = df_runs.sort_values(['metrics.test_auc'],ascending=False)
top_runs.head()

,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.train_auc,metrics.train_accuracy,metrics.test_auc,metrics.test_accuracy,tags.mlflow.log-model.history,tags.mlflow.user,tags.mlflow.source.name,tags.mlflow.source.type,tags.mlflow.runName
39,707f40bad8fb4e4e93c67c40d2d9cdbd,73,FINISHED,s3://models/73/707f40bad8fb4e4e93c67c40d2d9cdb...,2021-07-28 05:32:27.428000+00:00,2021-07-28 05:32:33.432000+00:00,1.0,1.0,0.928571,0.999375,"[{""run_id"": ""707f40bad8fb4e4e93c67c40d2d9cdbd""...",root,/opt/miniconda/lib/python3.7/site-packages/ipy...,LOCAL,GradientBoostingClassifier
27,b079e23bc82046fb96f406a69660da88,73,FINISHED,s3://models/73/b079e23bc82046fb96f406a69660da8...,2021-07-28 06:30:21.335000+00:00,2021-07-28 06:30:23.638000+00:00,1.0,1.0,0.928571,0.999375,"[{""run_id"": ""b079e23bc82046fb96f406a69660da88""...",root,/opt/miniconda/lib/python3.7/site-packages/ipy...,LOCAL,AdaBoostClassifier
1,da3352d8938342ba9e2ba051c42d3bdd,73,FINISHED,s3://models/73/da3352d8938342ba9e2ba051c42d3bd...,2021-07-29 08:22:47.834000+00:00,2021-07-29 08:22:49.525000+00:00,1.0,1.0,0.928571,0.999375,"[{""run_id"": ""da3352d8938342ba9e2ba051c42d3bdd""...",root,/opt/miniconda/lib/python3.7/site-packages/ipy...,LOCAL,AdaBoostClassifier
24,796278624dc0436eadbfed32052bd8e5,73,FINISHED,s3://models/73/796278624dc0436eadbfed32052bd8e...,2021-07-28 10:54:49.588000+00:00,2021-07-28 10:54:55.560000+00:00,1.0,1.0,0.928571,0.999375,"[{""run_id"": ""796278624dc0436eadbfed32052bd8e5""...",root,/opt/miniconda/lib/python3.7/site-packages/ipy...,LOCAL,GradientBoostingClassifier
29,283a35504d7442d19968b4a1ea427854,73,FINISHED,s3://models/73/283a35504d7442d19968b4a1ea42785...,2021-07-28 06:28:42.516000+00:00,2021-07-28 06:28:49.489000+00:00,1.0,1.0,0.928571,0.999375,"[{""run_id"": ""283a35504d7442d19968b4a1ea427854""...",root,/opt/miniconda/lib/python3.7/site-packages/ipy...,LOCAL,GradientBoostingClassifier


## Selecting best model

In [37]:
artifacts = top_runs.iloc[0]["artifact_uri"]
run_id = top_runs.iloc[0]["run_id"]
model_name = top_runs.iloc[0]["tags.mlflow.runName"] 


print('Best model_artifacts :',artifacts)
print("=" * 100)
print('Best model run_id :',run_id)
print("=" * 100)
print('Best model :',model_name)
print("=" * 100)
print("Best model experiment id :",exp_id)

Best model_artifacts : s3://models/73/707f40bad8fb4e4e93c67c40d2d9cdbd/artifacts
Best model run_id : 707f40bad8fb4e4e93c67c40d2d9cdbd
Best model : GradientBoostingClassifier
Best model experiment id : 73


## Registering best model

In [38]:
result = mlflow.register_model(
    "runs:/" + run_id,
    MODEL_NAME+'_'+model_name
)

Registered model 'cred_fraud_detection-model_GradientBoostingClassifier' already exists. Creating a new version of this model...
2021/07/29 08:22:57 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: cred_fraud_detection-model_GradientBoostingClassifier, version 3
Created version '3' of model 'cred_fraud_detection-model_GradientBoostingClassifier'.


In [39]:
print('Registered model information :')
print('=='*50)
print(pd.DataFrame(result, columns =['Attribute','Value']))

Registered model information :
                 Attribute                                              Value
0       creation_timestamp                                      1627546977902
1            current_stage                                               None
2              description                                                   
3   last_updated_timestamp                                      1627546977902
4                     name  cred_fraud_detection-model_GradientBoostingCla...
5                   run_id                   707f40bad8fb4e4e93c67c40d2d9cdbd
6                 run_link                                                   
7                   source  s3://models/73/707f40bad8fb4e4e93c67c40d2d9cdb...
8                   status                                              READY
9           status_message                                                   
10                    tags                                                 {}
11                 user_id       

In [40]:
print('Experiment information :')
print('=='*50)
print(pd.DataFrame(exp_details, columns =['Attribute','Value']))

Experiment information :
           Attribute                        Value
0  artifact_location               s3://models/73
1      experiment_id                           73
2    lifecycle_stage                       active
3               name  credit-card-fraud-detection
4               tags                           {}


## storing best model address for deployment

In [41]:

time = pd.to_datetime(datetime.now())

In [42]:
dictionary = [{"model":model_name,
             "location":artifacts+'/'+model_name,
             "run_id":run_id,
            'experiment_id':exp_id,
              'prod_time':time,
              'top_features':predictors,
              }]

In [43]:
dataset = pd.DataFrame(dictionary)
dataset

,model,location,run_id,experiment_id,prod_time,top_features
0,GradientBoostingClassifier,s3://models/73/707f40bad8fb4e4e93c67c40d2d9cdb...,707f40bad8fb4e4e93c67c40d2d9cdbd,73,2021-07-29 08:22:57.942855,"[Time, V1, V2, V3, V4, V5, V6, V7, V8, V9, V10..."


In [44]:
POSTGRES_PASSWORD = passwrd_parser(POSTGRES_PASSWORD)

In [45]:
write_to_feature_store(POSTGRES_USERNAME,POSTGRES_PASSWORD,POSTGRES_ADDRESS,
                       POSTGRES_PORT,POSTGRES_DBNAME,PROJECT_SCHEMA,dataset,TABLE_MODEL)